In [1]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path: sys.path.append("..")

In [2]:
import time
from src.classifier.agent import Agent
from src.utils.utils import create_directory
from src.utils.decorator import close_on_finish_decorator

In [3]:
import torch
def run_me(name,i_limit=5,base_config='base'):
    torch.cuda.empty_cache()
    agent = Agent(name, base_config=base_config)
    logged_metrics = []
    
    agent.load_model() 
    agent.load_trainer()
    print(f"{agent.trainer.logger.log_dir.rsplit('/',1)[0]}/")
    create_directory(f"{agent.trainer.logger.log_dir.rsplit('/',1)[0]}/") 
    with open(f"{agent.trainer.logger.log_dir.rsplit('/',1)[0]}/hparams.json", "w") as write_file:
        json.dump(agent._config, write_file, indent = 6)
            
    try:
        i = 0
        while True:
            agent.load_model() 
            agent.load_trainer()
            print(f"Dataloader fold: {agent.dataloader.kfold_index}")
            close_on_finish_decorator(agent.trainer.fit, agent.trainer.logger.log_dir, agent.model, datamodule=agent.dataloader, message=agent._config) 
            
            with open(agent.trainer.logger.log_dir.rsplit("/",1)[0] + "/logged_metrics", 'a+') as f:
                f.write(str(agent.trainer.logged_metrics))
                
            logged_metrics.append(agent.trainer.logged_metrics)
            time.sleep(10)
            i += 1
            #del tr

            
            if not agent.dataloader.next_fold() or i == i_limit and i_limit != -1: break
            torch.cuda.empty_cache()
    except Exception as e:
        torch.cuda.empty_cache()
        raise e
    print(f"{name}: {logged_metrics}")

In [12]:
models = ['vgg16']
for name in models:
    run_me(name, base_config='base',i_limit=6)

***Defined dataloader:***
Data directory: data/SPM_categorised/ADNI1/
Dataset sizes - Training: 273 Validation: 31
Seed: 0
Augmentation: Enabled
KFold: Enabled - Fold: 1/10

Architecture [VGG] was created
***Defined hyperparameters:***
"arch":                {'name': 'vgg16', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       None
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': False}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": normal


/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/final/vgg16/20210528193314


Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
/var/metrics/codetests/logs/tb/final/vgg16/20210528193314


FileNotFoundError: [Errno 2] No such file or directory: '/var/metrics/codetests/logs/tb/final/vgg16/20210528193314/hparams.json'

In [ ]:
models = ['customCNN']
for name in models:
    run_me(name, base_config='augmented',i_limit=-1)

/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/augmented/customCNN/20210528202437
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


***Defined dataloader:***
Data directory: data/SPM_categorised/ADNI1/
Dataset sizes - Training: 273 Validation: 31
Seed: 0
Augmentation: Disabled
KFold: Enabled - Fold: 1/10

Architecture [customCNN] was created
***Defined hyperparameters:***
"arch":                {'name': 'customCNN', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       None
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': False}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": normal
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
/var/metrics/codetests/logs/tb/augmented/customCNN/20210528202437/
Architecture [customCNN] was created
***Defined hyperparam


  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | customCNN        | 5.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
5.3 M     Trainable params
0         Non-trainable params
5.3 M     Total params
21.285    Total estimated model params size (MB)


Metric loss/val improved. New best score: 1.031


Metric loss/val improved by 0.006 >= min_delta = 0.0001. New best score: 1.025


...

Architecture [customCNN] was created
***Defined hyperparameters:***
"arch":                {'name': 'customCNN', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       None
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': False}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": normal
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
Dataloader fold: 8


Metric loss/val improved. New best score: 1.027


Metric loss/val improved by 0.005 >= min_delta = 0.0001. New best score: 1.022


Metric loss/val improved by 0.009 >= min_delta = 0.0001. New best score: 1.012


Metric loss/val improved by 0.000 >= min_delta = 0.0001. New best score: 1.012


In [ ]:
models = ['customCNN']
for name in models:
    run_me(name, base_config='no_augmented',i_limit=-1)

In [ ]:
models = ['resnet18_brew2']
for name in models:
    run_me(name, base_config='augmented',i_limit=-1)

In [ ]:
models = ['resnet18_brew2']
for name in models:
    run_me(name, base_config='no_augmented',i_limit=-1)

In [ ]:
models = ['vgg16','resnet18_brew2','resnet50','customCNN']
for name in models:
    run_me(name, base_config='base2',i_limit=-1)

***Defined dataloader:***
Data directory: data/SPM_categorised/AIH/
Dataset sizes - Training: 421 Validation: 47
Seed: 0
Augmentation: Disabled
KFold: Enabled - Fold: 1/10



/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/final/base2/resnet18_brew2/20210524235112
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 8.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.154    Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew2', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([0.9953, 1.0024, 1.0024], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
Dataloader fold: 1


Metric loss/val improved. New best score: 1.265


Metric loss/val improved by 0.118 >= min_delta = 0.0001. New best score: 1.147


...

Metric loss/val improved by 0.018 >= min_delta = 0.0001. New best score: 0.797


Monitored metric loss/val did not improve in the last 20 records. Best score: 0.797. Signaling Trainer to stop.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 8.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.154    Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew2', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([0.9953, 1.0024, 1.0024], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
Dataloader fold: 9


Metric loss/val improved. New best score: 1.136


Metric loss/val improved by 0.029 >= min_delta = 0.0001. New best score: 1.107


In [ ]:
models = ['vgg16','vgg11_brew','resnet50', 'resnet18_brew2','customCNN']
for name in models:
    run_me(name, base_config='base',i_limit=6)

In [ ]:
,''

In [ ]:
models = ['resnet18_brew2','amira_vgg', 'customCNN','resnet50','vgg11_brew','vgg16']
for name in models:
    run_me(name, base_config='base', i_limit=5)

***Defined dataloader:***
Data directory: data/SPM_categorised/ADNI1/
Dataset sizes - Training: 273 Validation: 31
Seed: 0
Augmentation: Disabled
KFold: Enabled - Fold: 1/10



/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/resnet18_brew2/20210521172659
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 8.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.154    Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew2', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
Dataloader fold: 1


Metric loss/val improved. New best score: 1.293


Metric loss/val improved by 0.027 >= min_delta = 0.0001. New best score: 1.266


...

Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew2', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
Dataloader fold: 3


Metric loss/val improved. New best score: 1.227


Metric loss/val improved by 0.007 >= min_delta = 0.0001. New best score: 1.220


Metric loss/val improved by 0.053 >= min_delta = 0.0001. New best score: 1.167


Metric loss/val improved by 0.065 >= min_delta = 0.0001. New best score: 1.102


In [ ]:
models = ['resnet50','vgg11_brew','vgg16','resnet18_brew2','customCNN',]
for name in models:
    run_me(name, base_config='base',i_limit=6)

In [4]:
models = ['debug']
for name in models:
    run_me(name, base_config='base',i_limit=-1)

***Defined dataloader:***
Data directory: data/SPM_categorised/ADNI1/
Dataset sizes - Training: 273 Validation: 31
Seed: 0
Augmentation: Enabled
KFold: Enabled - Fold: 1/10



/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/debug/20210521160133
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 8.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.358    Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 1
{'epoch': tensor(4.), 'loss/train': tensor(1.0987, device='cuda:0'), 'loss/val': tensor(1.0569), 'Accuracy/val': tensor(0.4839), 'Precision/val': tensor(0.4839), 'Recall/val': tensor(0.4839), 'AUROC/val': tensor(0.6461), 'Specificity/val': tensor(0.7

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 8.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.358    Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 2
{'epoch': tensor(4.), 'loss/train': tensor(1.0840, device='cuda:0'), 'loss/val': tensor(1.3403), 'Accuracy/val': tensor(0.2258), 'Precision/val': tensor(0.2258), 'Recall/val': tensor(0.2258), 'AUROC/val': tensor(0.6673), 'Specificity/val': tensor(0.6

...

Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 8
{'epoch': tensor(4.), 'loss/train': tensor(1.1112, device='cuda:0'), 'loss/val': tensor(1.1325), 'Accuracy/val': tensor(0.2581), 'Precision/val': tensor(0.2581), 'Recall/val': tensor(0.2581), 'AUROC/val': tensor(0.6143), 'Specificity/val': tensor(0.6

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 8.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.358    Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 9
{'epoch': tensor(4.), 'loss/train': tensor(1.0731, device='cuda:0'), 'loss/val': tensor(1.1483), 'Accuracy/val': tensor(0.2903), 'Precision/val': tensor(0.2903), 'Recall/val': tensor(0.2903), 'AUROC/val': tensor(0.6361), 'Specificity/val': tensor(0.6

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 8.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.358    Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 10
{'epoch': tensor(4.), 'loss/train': tensor(1.0478, device='cuda:0'), 'loss/val': tensor(1.0345), 'Accuracy/val': tensor(0.4194), 'Precision/val': tensor(0.4194), 'Recall/val': tensor(0.4194), 'AUROC/val': tensor(0.7012), 'Specificity/val': tensor(0.

In [ ]:
models = ['resnet18_brew2']
for name in models:
    run_me(name, base_config='base',i_limit=-1)

In [ ]:
models = ['customCNN']
for name in models:
    run_me(name, base_config='base',i_limit=-1)

***Defined dataloader:***
Data directory: data/SPM_categorised/ADNI1/
Dataset sizes - Training: 273 Validation: 31
Seed: 0
Augmentation: Enabled
KFold: Enabled - Fold: 1/10



/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/customCNN/20210521152336
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | customCNN        | 5.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
5.3 M     Trainable params
0         Non-trainable params
5.3 M     Total params
21.285    Total estimated model params size (MB)


Architecture [customCNN] was created
***Defined hyperparameters:***
"arch":                {'name': 'customCNN', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 1


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | customCNN        | 5.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
5.3 M     Trainable params
0         Non-trainable params
5.3 M     Total params
21.285    Total estimated model params size (MB)


Architecture [customCNN] was created
***Defined hyperparameters:***
"arch":                {'name': 'customCNN', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 1e-06, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 2


In [4]:
models = ['resnet50','resnet18_brew2','amira_vgg','vgg11_brew','vgg16']
for name in models:
    run_me(name, base_config='base',i_limit=-1)

***Defined dataloader:***
Data directory: data/SPM_categorised/ADNI1/
Dataset sizes - Training: 273 Validation: 31
Seed: 420
Augmentation: Enabled
KFold: Enabled - Fold: 1/10



/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/resnet50/20210520184505
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 46.2 M
1 | criteria           | CrossEntropyLoss | 0     
2 | train_metrics      | MetricCollection | 0     
3 | valid_metrics      | MetricCollection | 0     
4 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
46.2 M    Trainable params
0         Non-trainable params
46.2 M    Total params
184.645   Total es

Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet50', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.02, 'weight_decay': 0, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
Dataloader fold: 1


Metric loss/val improved. New best score: 933724.500


Metric loss/val improved by 933705.000 >= min_delta = 0.0001. New best score: 19.511


...

Metric loss/val improved by 0.538 >= min_delta = 0.0001. New best score: 1.064


torch.Size([8, 512, 2, 2, 2])
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])7'}
torch.Size([8, 512, 2, 2, 2])6'}
torch.Size([8, 512, 2, 2, 2])5'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2, 2, 2])5'}
torch.Size([8, 512, 2, 2, 2])5'}
torch.Size([8, 512, 2, 2, 2])6'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2, 2, 2])5'}
torch.Size([8, 512, 2, 2, 2])5'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2,

Metric loss/val improved by 0.003 >= min_delta = 0.0001. New best score: 1.061


torch.Size([8, 512, 2, 2, 2])
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])2'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])4'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2, 2, 2])3'}
torch.Size([8, 512, 2,

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [4]:
models = ['resnet18_brew2','resnet50','vgg11_brew','vgg16','resnet18','amira_vgg']
for name in models:
    run_me(name, base_config='base',i_limit=6)

***Defined dataloader:***
Data directory: data/SPM_categorised/ADNI1/
Dataset sizes - Training: 273 Validation: 31
Seed: 0
Augmentation: Enabled
KFold: Enabled - Fold: 1/10



/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/resnet18_brew2/20210521193407
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 8.3 M 
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
8.3 M     Trainable params
0         Non-trainable params
8.3 M     Total params
33.154    Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18_brew2', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 0, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
Dataloader fold: 1


Metric loss/val improved. New best score: 1.101


Metric loss/val improved by 0.055 >= min_delta = 0.0001. New best score: 1.046


...

Monitored metric loss/val did not improve in the last 20 records. Best score: 1.006. Signaling Trainer to stop.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | ResNet           | 33.2 M
1 | criteria           | CrossEntropyLoss | 0     
2 | valid_metrics      | MetricCollection | 0     
3 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
33.2 M    Trainable params
0         Non-trainable params
33.2 M    Total params
132.646   Total estimated model params size (MB)


Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet18', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.2466, 0.6500, 1.5167], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 0, 'amsgrad': False}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": None
Enabled callbacks:  ['LitProgressBar', 'EarlyStopping']
Dataloader fold: 4


Metric loss/val improved. New best score: 1.670


Metric loss/val improved by 0.691 >= min_delta = 0.0001. New best score: 0.979


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
models = ['amira_vgg']
for name in models:
    run_me(name, base_config='base',i_limit=6)

***Defined dataloader:***
Data directory: data/SPM_categorised/Amira/
Dataset sizes - Training: 500 Validation: 56
Seed: 420
Augmentation: Enabled
KFold: Enabled - Fold: 1/10

Architecture [amira_vgg] was created
***Defined hyperparameters:***
"arch":                {'name': 'amira_vgg', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.1905, 0.9259, 0.9259], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 0, 'amsgrad': True}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": normal


/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Missing logger folder: /var/metrics/codetests/logs/tb/amira_vgg/20210518170258
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | amira_vgg        | 37.0 M
1 | criteria           | CrossEntropyLoss | 0     
2 | train_metrics      | MetricCollection | 0     
3 | valid_metrics      | MetricCollection | 0     
4 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
37.0 M    Trainable params
0         Non-trainable params
37.0 M    Total params
148.174   Total e

Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 1
Architecture [amira_vgg] was created
***Defined hyperparameters:***
"arch":                {'name': 'amira_vgg', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.1905, 0.9259, 0.9259], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 0, 'amsgrad': True}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": normal


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | amira_vgg        | 37.0 M
1 | criteria           | CrossEntropyLoss | 0     
2 | train_metrics      | MetricCollection | 0     
3 | valid_metrics      | MetricCollection | 0     
4 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
37.0 M    Trainable params
0         Non-trainable params
37.0 M    Total params
148.174   Total estimated model params size (MB)


Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 2
Architecture [amira_vgg] was created
***Defined hyperparameters:***
"arch":                {'name': 'amira_vgg', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.1905, 0.9259, 0.9259], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 0, 'amsgrad': True}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": normal


...

Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 3
Architecture [amira_vgg] was created
***Defined hyperparameters:***
"arch":                {'name': 'amira_vgg', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.1905, 0.9259, 0.9259], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 0, 'amsgrad': True}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": normal


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | amira_vgg        | 37.0 M
1 | criteria           | CrossEntropyLoss | 0     
2 | train_metrics      | MetricCollection | 0     
3 | valid_metrics      | MetricCollection | 0     
4 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
37.0 M    Trainable params
0         Non-trainable params
37.0 M    Total params
148.174   Total estimated model params size (MB)


Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 4
Architecture [amira_vgg] was created
***Defined hyperparameters:***
"arch":                {'name': 'amira_vgg', 'args': {'num_channels': 1, 'num_classes': 3}}
"checkpoint_path":     None
"class_weights":       tensor([1.1905, 0.9259, 0.9259], device='cuda:0')
"hp_metrics":          ['loss/train', 'loss/val', 'Accuracy/val', 'AUROC/val', 'Precision/val', 'Recall/val', 'Specificity/val']
"loss":                {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0001, 'weight_decay': 0, 'amsgrad': True}}
"roi_hparams":         {'enable': False, 'output_shape': [3, 3, 3], 'boundary_boxes': []}
"weight_distribution": normal


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | model              | amira_vgg        | 37.0 M
1 | criteria           | CrossEntropyLoss | 0     
2 | train_metrics      | MetricCollection | 0     
3 | valid_metrics      | MetricCollection | 0     
4 | valid_dummy_metric | AUROC            | 0     
--------------------------------------------------------
37.0 M    Trainable params
0         Non-trainable params
37.0 M    Total params
148.174   Total estimated model params size (MB)


Enabled callbacks:  ['LitProgressBar']
Dataloader fold: 5
